# Basic Federated Learning Example

In this notebook, we show how PySyft can be used to train a model using Federated Learning. We train a simple linear model using stochastic gradient descent across a toy dataset with two different owners, Alice and Bob.

This notebook is also available on Colab: https://colab.research.google.com/drive/1F3ALlA3ogfeeVXuwQwVoX4PimzTDJhPy

This notebook was created in the following video (at the end): https://www.twitch.tv/videos/275910219


# Step 1: Hook PyTorch and Create Workers

In [1]:
import syft as sy
from syft import Variable as Var
from syft import nn
from syft import optim

# this is our hook
hook = sy.TorchHook()
me = hook.local_worker

bob = sy.VirtualWorker(id="bob",hook=hook)
alice = sy.VirtualWorker(id="alice",hook=hook)

me.add_worker(bob)
me.add_worker(alice)

# Step 2: Initialize Dataset and Move to Alice and Bob

In [2]:
# create our dataset
data = Var(sy.FloatTensor([[0,0],[0,1],[1,0],[1,1]]))
target = Var(sy.FloatTensor([[0],[0],[1],[1]]))

data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:].send(alice)
target_alice = target[2:].send(alice)

# Step 3: Create our Model

In [3]:
# create our model
model = nn.Linear(2,1)

In [4]:
opt = optim.SGD(params=model.parameters(),lr=0.1)

# Step 4: Train over Distributed Dataset

(this is the Federated Learning part)

In [5]:
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

In [6]:
for iter in range(20):

    for data,target in datasets:
        model.send(data.location)

        # update the model
        opt.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        model.get()
        opt.step()

        print(loss.get().data[0])

1.6867214441299438
5.170070171356201
0.3087354898452759
0.24715974926948547
0.2646021544933319
0.09975367039442062
0.16370245814323425
0.057370081543922424
0.09991589188575745
0.03417927771806717
0.06139484792947769
0.02060863748192787
0.03806630149483681
0.012580597773194313
0.023840248584747314
0.007791425101459026
0.015094665810465813
0.004905462730675936
0.009669410064816475
0.0031449540983885527
0.006270182318985462
0.0020553572103381157
0.004117220640182495
0.0013697484973818064
0.002737795002758503
0.0009303892147727311
0.0018432490760460496
0.0006433306261897087
0.0012559160823002458
0.0004520445363596082
0.0008654575794935226
0.0003221090300939977
0.0006026809569448233
0.000232243663049303
0.00042372927418909967
0.0001690747303655371
0.0003004876198247075
0.00012404433800838888
0.00021472539810929447
9.15599666768685e-05
